In [25]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [26]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp_2022/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['jan', 'feb', 'mar']
sheet = 'Sheet'

In [27]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [28]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[(df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index().groupby('snils').max().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[(sums.stv >= 1.0) & (sums.status == 'Работа') & (sums.day == 1)]
    sums = sums[['inn_y', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_y', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_y': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [29]:
def create_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data(months[i])
        else:
            res = pd.merge(res, load_data(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [30]:
file = 'mar'
d = pd.read_excel(path_to_data + file + '.xlsx')

In [31]:
d.stv = d.apply(lambda row: to_double(row, 'stv'), axis=1)

In [32]:
s = d[~(d.type == 'Внешнее совместительство')]
ss = s.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()

In [33]:
ss = pd.merge(ss, s[s.type == 'Основное место работы'], how='left', on=['inn', 'snils'])

In [34]:
ss[ss.job.isna()]

,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
4848,5051005670,013-184-668 28,9333.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4865,5051005670,014-688-674 79,23100.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5164,5051005670,073-954-680 04,21000.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5287,5051005670,117-332-744 39,9333.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6615,5074045703,038-825-734 88,10400.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
165376,9715206976,156-020-820 28,6000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165405,9715206976,166-110-242 25,46363.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165425,9715206976,171-990-672 03,26250.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165434,9715206976,174-534-080 69,21050.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
ss.dropna(how='all', subset=['job'])[ss.sum_x != ss.sum_y]

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_14624\1276137153.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ss.dropna(how='all', subset=['job'])[ss.sum_x != ss.sum_y]


,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
718,5003021368,154-314-049 32,0.00,NaN,Учитель,0.0,Основное место работы,1.00,Увольнение,Увольнение
975,5003021495,001-703-734 92,59925.00,52663.00,Помощник воспитателя,1.0,Основное место работы,1.00,Работа,Работа
1011,5003021495,010-374-085 98,75003.80,64110.80,Уборщик служебных и производственных помещений,1.0,Основное место работы,1.00,Работа,Работа
1017,5003021495,011-347-332 98,71819.97,58860.72,Музыкальный руководитель,1.0,Основное место работы,1.00,Работа,Работа
1029,5003021495,014-158-355 22,60830.09,53568.09,Рабочий по комплексному обслуживанию и ремонту...,1.0,Основное место работы,1.00,Работа,Работа
...,...,...,...,...,...,...,...,...,...,...
165487,9715206976,204-248-388 42,115000.00,85500.00,Учитель-логопед,1.0,Основное место работы,1.00,Работа,Работа
165488,9715206976,204-279-880 68,116937.50,96550.00,Специалист по сетевому администрированию,1.0,Основное место работы,1.00,Работа,Работа
165494,9715206976,210-512-230 81,130065.11,96415.11,Учитель,1.0,Основное место работы,1.05,Работа,Работа
165495,9715206976,212-203-372 89,24816.08,24066.08,Педагог дополнительного образования,0.0,Основное место работы,1.00,Отпуск неоплачиваемый по разрешению работодателя,Работа


In [36]:
ss

,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
0,5003021368,001-171-362 75,61500.00,61500.00,Помощник воспитателя,1.0,Основное место работы,1.0,Работа,Работа
1,5003021368,001-191-281 85,82600.36,82600.36,Учитель,0.0,Основное место работы,1.0,Работа,Работа
2,5003021368,001-244-107 65,109077.82,109077.82,Учитель,1.0,Основное место работы,1.0,Работа,Работа
3,5003021368,001-263-234 77,87744.00,87744.00,Воспитатель,1.0,Основное место работы,1.0,Работа,Работа
4,5003021368,001-322-594 80,81798.18,81798.18,Специалист,1.0,Основное место работы,1.0,Работа,Работа
...,...,...,...,...,...,...,...,...,...,...
166739,9718071371,196-882-391 45,77596.00,77596.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
166740,9718071371,198-036-430 93,110851.00,110851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
166741,9718071371,199-418-719 35,145851.00,145851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
166742,9718071371,205-412-630 13,189010.00,189010.00,Начальник группы,1.0,Основное место работы,1.0,Работа,Работа


In [37]:
def load_data_all(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[~(df.type == 'Внешнее совместительство')]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index().groupby('snils').max().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[['inn_y', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_y', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_y': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [38]:
def create_full_res(months):
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = load_data_all(months[i])
        else:
            res = pd.merge(res, load_data_all(months[i]), how='outer', on=['inn', 'snils'])
    return res

In [39]:
res = create_res(months)

In [40]:
res[res.job_mar.isna() == False]

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar
1,7.736615e+09,001-010-312 25,164214.30,1.0,Учитель,146547.14,1.0,Учитель,160599.30,1.0,Учитель
2,7.723170e+09,001-011-996 67,161441.63,1.0,Учитель,167905.34,1.0,Учитель,166752.15,1.0,Учитель
3,7.727124e+09,001-017-453 65,90135.84,1.0,Документовед,87135.84,1.0,Секретарь руководителя,86627.92,1.0,Секретарь руководителя
4,7.733089e+09,001-017-484 72,88112.10,1.0,Учитель,NaN,NaN,NaN,86787.10,1.0,Учитель
5,7.729776e+09,001-020-711 41,65993.00,1.0,Воспитатель,60020.00,1.0,Воспитатель,68000.00,1.0,Воспитатель
...,...,...,...,...,...,...,...,...,...,...,...
153540,7.734409e+09,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,1.0,Специалист
153541,7.736615e+09,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,1.0,Уборщик территории
153542,7.743021e+09,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...
153543,7.736051e+09,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...


In [41]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [43]:
import pathlib

In [48]:
save_path = str(pathlib.Path().resolve()) + '\\re_1_q\\'

In [50]:
print_df(res[(res.inn == 5003021368) & (res.job_feb == 'Учитель')], save_path + 'teach_check')

In [51]:
res[(res.inn == 9718071371)]

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar
62,9.718071e+09,001-102-623 46,147930.0,1.0,ведущий Бухгалтер,157930.00,1.0,ведущий Бухгалтер,NaN,NaN,NaN
267,9.718071e+09,001-184-402 83,177930.0,1.0,ведущий Бухгалтер,177930.00,1.0,ведущий Бухгалтер,232930.00,1.0,ведущий Бухгалтер
381,9.718071e+09,001-200-216 33,115851.0,1.0,Бухгалтер,142896.32,1.0,Бухгалтер,177930.00,1.0,ведущий Бухгалтер
435,9.718071e+09,001-208-598 92,96078.0,1.0,Бухгалтер,117705.00,1.0,Бухгалтер,127705.00,1.0,Бухгалтер
682,9.718071e+09,001-257-058 90,114851.0,1.0,Бухгалтер,110851.00,1.0,Бухгалтер,110851.00,1.0,Бухгалтер
...,...,...,...,...,...,...,...,...,...,...,...
151858,9.718071e+09,174-487-457 12,NaN,NaN,NaN,NaN,NaN,NaN,527029.00,1.0,Начальник управления
152289,9.718071e+09,180-190-843 58,NaN,NaN,NaN,NaN,NaN,NaN,36504.82,1.0,Бухгалтер
152379,9.718071e+09,181-370-565 64,NaN,NaN,NaN,NaN,NaN,NaN,197930.00,1.0,Ведущий специалист
152517,9.718071e+09,183-851-618 98,NaN,NaN,NaN,NaN,NaN,NaN,120851.00,1.0,Экономист


In [52]:
def szp(row, months):
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    cnt = 0
    sum_szp = 0
    for month in months:
        if pd.isna(row['sum_' + month]):
            continue
        cnt += 1
        sum_szp += row['sum_' + month]
    # if row['inn'] == 9715217689 and row['snils'] == '173-260-871 66':
    #     print(cnt, sum_szp)
    return sum_szp / cnt

In [53]:
res['szp'] = res.apply(lambda row: szp(row, months), axis= 1)

In [54]:
depr_month_inc = 84100.0

In [55]:
df = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='УГД')

In [56]:
df

,Должность,УГД,Педагогический,Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми,общее образование,доп обр,прочий
0,Администратор,Прочий персонал (включая иной административный...,0,0,0,0,1
1,Администратор баз данных,Прочий персонал (включая иной административный...,0,0,0,0,1
2,Аккомпаниатор-концертмейстер,Прочий персонал (включая иной административный...,0,0,0,0,1
3,Аналитик,Прочий персонал (включая иной административный...,0,0,0,0,1
4,Архивариус,Прочий персонал (включая иной административный...,0,0,0,0,1
...,...,...,...,...,...,...,...
300,Экскурсовод,Прочий персонал (включая иной административный...,0,0,0,0,1
301,Эксперт,Прочий персонал (включая иной административный...,0,0,0,0,1
302,Электромеханик,Прочий персонал (включая иной административный...,0,0,0,0,1
303,Электромонтер по ремонту и обслуживанию электр...,Прочий персонал (включая иной административный...,0,0,0,0,1


In [57]:
ped = df[df['Педагогический'] == 1]['Должность'].tolist()
ped

['Ассистент',
 'Воспитатель',
 'Декан факультета',
 'Директор института',
 'Доцент',
 'Заведующий кафедрой',
 'Инструктор по труду',
 'Инструктор по физической культуре',
 'Инструктор-методист',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Методист',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-библиотекарь',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель',
 'Преподаватель ',
 'Преподаватель-организатор основ безопасности жизнедеятельности',
 'Профессор',
 'Руководитель физического воспитания',
 'Социальный педагог',
 'Старший вожатый',
 'Старший воспитатель',
 'Старший инструктор-методист',
 'Старший методист',
 'Старший педагог дополнительного образования',
 'Старший педагог-психолог',
 'Старший преподаватель',
 'Старший тренер-преподаватель',
 'Тренер-преподаватель',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед']

In [58]:
isp = df[df['Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми'] == 1]['Должность'].tolist()
isp

['Воспитатель',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Младший воспитатель',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-библиотекарь',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Помощник воспитателя',
 'Преподаватель',
 'Преподаватель ',
 'Социальный педагог',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед']

In [59]:
vosp = ['Старший воспитатель', 'Воспитатель']

In [60]:
def szp_ped(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ped:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum_szp / cnt

In [61]:
def szp_teach(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] != 'Учитель':
        return np.nan
    return sum_szp / cnt

In [62]:
def szp_vosp(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in vosp:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] not in vosp:
        return np.nan
    return sum_szp / cnt

In [63]:
res['szp_ped'] = res.apply(lambda row: szp_ped(row, months), axis=1)

In [64]:
res['szp_teach'] = res.apply(lambda row: szp_teach(row, months), axis=1)

In [65]:
res['szp_vosp'] = res.apply(lambda row: szp_vosp(row, months), axis=1)

In [66]:
buckets_teach_old = [(0, 80), (80, 90), (90, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
buckets_teach_new = [(0, 90), (90, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
buckets_vosp_old = [(0, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
buckets_vosp_new = [(0, 60), (60, 80), (80, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 1000)]
teach_szp_levels_old = {}
teach_szp_levels_new = {}
vosp_szp_levels_old = {}
vosp_szp_levels_new = {}

In [67]:
print_df(temp, save_path + 'temp')

NameError: name 'temp' is not defined

In [ ]:
temp

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach
0,7.720259e+09,001-005-586 64,124968.88,1.0,Заведующий хозяйством,140968.88,1.0,Заведующий хозяйством,60434.74,1.0,Заведующий хозяйством,NaN,NaN
1,7.718310e+09,001-005-786 70,61833.04,1.0,Учитель,55448.10,1.0,Учитель,85809.12,1.0,Учитель,NaN,67696.753333
2,7.735601e+09,001-008-669 78,66928.99,1.0,Учитель,53228.87,1.0,Учитель,75124.20,1.0,Учитель,NaN,65094.020000
3,7.736615e+09,001-010-312 25,146547.14,1.0,Учитель,164214.30,1.0,Учитель,160599.30,1.0,Учитель,NaN,157120.246667
4,7.723170e+09,001-011-996 67,167905.34,1.0,Учитель,161441.63,1.0,Учитель,166752.15,1.0,Учитель,NaN,165366.373333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176839,NaN,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,NaN,NaN,NaN,NaN
176840,NaN,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,NaN,NaN,NaN,NaN
176841,NaN,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,NaN,NaN,NaN,NaN
176842,NaN,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,NaN,NaN,NaN,NaN


In [68]:
res[res.szp_teach > 150000]

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach,szp_vosp
1,7.736615e+09,001-010-312 25,164214.30,1.0,Учитель,146547.14,1.0,Учитель,160599.30,1.0,Учитель,157120.246667,157120.246667,157120.246667,NaN
2,7.723170e+09,001-011-996 67,161441.63,1.0,Учитель,167905.34,1.0,Учитель,166752.15,1.0,Учитель,165366.373333,165366.373333,165366.373333,NaN
87,7.702833e+09,001-131-908 67,152088.02,1.0,Учитель,160320.62,1.0,Учитель,175707.24,1.0,Учитель,162705.293333,162705.293333,162705.293333,NaN
106,7.724189e+09,001-147-987 10,157358.47,1.0,Учитель,NaN,NaN,NaN,147832.41,1.0,Учитель,152595.440000,152595.440000,152595.440000,NaN
141,7.736051e+09,001-163-929 95,212679.15,1.0,Учитель,NaN,NaN,NaN,230903.40,1.0,Учитель,221791.275000,221791.275000,221791.275000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153469,7.729783e+09,209-661-850 84,NaN,NaN,NaN,NaN,NaN,NaN,157003.58,1.0,Учитель,157003.580000,157003.580000,157003.580000,NaN
153471,7.731582e+09,209-723-071 59,NaN,NaN,NaN,NaN,NaN,NaN,167283.07,1.0,Учитель,167283.070000,167283.070000,167283.070000,NaN
153473,7.713783e+09,210-219-032 87,NaN,NaN,NaN,NaN,NaN,NaN,164441.03,1.0,Учитель,164441.030000,164441.030000,164441.030000,NaN
153505,7.729413e+09,213-011-842 90,NaN,NaN,NaN,NaN,NaN,NaN,168718.64,1.0,Учитель,168718.640000,168718.640000,168718.640000,NaN


In [69]:
for bucket in buckets_teach_old:
    teach_szp_levels_old[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_teach'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
teach_szp_levels_old

NameError: name 'temp' is not defined

In [ ]:
for bucket in buckets_teach_new:
    teach_szp_levels_new[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_teach'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
teach_szp_levels_new

{'0-90': 8315,
 '90-100': 6703,
 '100-110': 7480,
 '110-120': 7932,
 '120-130': 7496,
 '130-140': 6213,
 '140-150': 4678,
 '150-1000': 7918}

In [ ]:
for bucket in buckets_vosp_old:
    vosp_szp_levels_old[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_vosp'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
vosp_szp_levels_old

{'0-50': 2369,
 '50-60': 5050,
 '60-70': 7433,
 '70-80': 6601,
 '80-90': 4175,
 '90-100': 2516,
 '100-110': 1448,
 '110-120': 826,
 '120-130': 500,
 '130-140': 295,
 '140-150': 144,
 '150-1000': 260}

In [ ]:
for bucket in buckets_vosp_new:
    vosp_szp_levels_new[str(bucket[0]) + '-' + str(bucket[1])] = temp[temp['szp_vosp'].between(bucket[0] * 1000, bucket[1] * 1000, inclusive='left')]['snils'].count()
vosp_szp_levels_new

{'0-60': 7339,
 '60-80': 14018,
 '80-100': 6691,
 '100-110': 1445,
 '110-120': 829,
 '120-130': 500,
 '130-140': 296,
 '140-150': 144,
 '150-1000': 260}

In [ ]:
print_df(temp[temp.szp_vosp.isna() == False][['inn', 'snils', 'stv_jan', 'stv_feb', 'stv_mar', 'job_mar', 'szp_vosp']], save_path + 'Воспитатели')

In [ ]:
temp

,inn,snils,sum_feb,stv_feb,job_feb,sum_jan,stv_jan,job_jan,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach
0,7.720259e+09,001-005-586 64,124968.88,1.0,Заведующий хозяйством,140968.88,1.0,Заведующий хозяйством,60434.74,1.0,Заведующий хозяйством,NaN,NaN
1,7.718310e+09,001-005-786 70,61833.04,1.0,Учитель,55448.10,1.0,Учитель,85809.12,1.0,Учитель,NaN,67696.753333
2,7.735601e+09,001-008-669 78,66928.99,1.0,Учитель,53228.87,1.0,Учитель,75124.20,1.0,Учитель,NaN,65094.020000
3,7.736615e+09,001-010-312 25,146547.14,1.0,Учитель,164214.30,1.0,Учитель,160599.30,1.0,Учитель,NaN,157120.246667
4,7.723170e+09,001-011-996 67,167905.34,1.0,Учитель,161441.63,1.0,Учитель,166752.15,1.0,Учитель,NaN,165366.373333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176757,7.734409e+09,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,1.0,Специалист,NaN,NaN
176758,7.736615e+09,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,1.0,Уборщик территории,NaN,NaN
176759,7.743021e+09,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN
176760,7.736051e+09,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN


In [70]:
res[res.snils == '047-832-306 66']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach,szp_vosp
45384,9.718071e+09,047-832-306 66,469642.0,1.0,Первый заместитель начальника,NaN,NaN,NaN,494642.0,1.0,Первый заместитель начальника,482142.0,NaN,NaN,NaN


In [71]:
val_5 = res[['inn', 'szp_ped']].groupby('inn').mean()
val_5['value'] = val_5['szp_ped'] / depr_month_inc
val_5

,szp_ped,value
inn,,
5.003021e+09,102092.215707,1.213938
5.003021e+09,111233.681474,1.322636
5.003022e+09,111311.665866,1.323563
5.003096e+09,104567.016461,1.243365
5.030032e+09,108526.287245,1.290443
...,...,...
7.743941e+09,129015.872016,1.534077
9.705102e+09,NaN,NaN
9.715207e+09,116005.620687,1.379377


In [72]:
print_df(val_5, save_path + 'val_5')
print_df(res[res.szp_ped.isna() == False][['inn', 'snils', 'szp_ped']], save_path + 'val_5_ppl')

In [73]:
val_6 = res[['inn', 'szp_ped']].groupby('inn').quantile(0.5).rename(columns={'szp_ped': 'q'})
val_6 = pd.merge(val_6, res[['inn', 'snils', 'szp_ped']], how='right', on='inn')
val_6_out = val_6
val_6 = val_6[val_6.szp_ped <= val_6.q][['inn', 'szp_ped']].groupby('inn').mean()
val_6

,szp_ped
inn,
5.003021e+09,75781.402741
5.003021e+09,85117.003186
5.003022e+09,80464.274085
5.003096e+09,74454.000225
5.030032e+09,75085.680529
...,...
7.743937e+09,96833.393949
7.743939e+09,71694.986000
7.743941e+09,101642.756214


In [74]:
print_df(val_6, save_path + 'val_6')
print_df(val_6_out, save_path + 'val_6_ppl')

In [75]:
res[res.snils == '154-824-084 70']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach,szp_vosp
97840,7.715311e+09,154-824-084 70,95508.44,1.0,Учитель,95227.31,1.0,Учитель,94743.63,1.0,Учитель,95159.793333,95159.793333,95159.793333,NaN


In [76]:
val_7 = pd.merge(res[['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt1'}), 
res[res.szp_teach >= 84100][['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt2'}), how='outer', on='inn')
val_7['val'] = val_7.cnt2 / val_7.cnt1
val_7[val_7.val < 1]

,cnt1,cnt2,val
inn,,,
5.003021e+09,296,291.0,0.983108
5.003021e+09,348,347.0,0.997126
5.030032e+09,66,65.0,0.984848
5.074019e+09,103,102.0,0.990291
5.074019e+09,86,85.0,0.988372
...,...,...,...
7.743021e+09,55,54.0,0.981818
7.743905e+09,97,95.0,0.979381
7.743937e+09,28,27.0,0.964286


In [77]:
print_df(val_7, save_path + 'val_7')
print_df(res[res.szp_teach.isna() == False][['inn', 'snils', 'szp_teach']], save_path + 'val_7_ppl')

In [78]:
def q_cnt1(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] <= row['0.1'] else np.nan

In [79]:
def q_cnt2(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] >= row['0.9'] else np.nan

In [80]:
q01 = res[['inn','szp']].groupby('inn').quantile(0.1).rename(columns={'szp':'0.1'})
q09 = res[['inn','szp']].groupby('inn').quantile(0.9).rename(columns={'szp':'0.9'})
q = pd.merge(q01, q09, how='right', on='inn')
qm = pd.merge(res[['inn', 'szp']], q, how='left', on='inn')
qm['cnt1'] = qm.apply(lambda row: q_cnt1(row), axis=1)
qm['cnt2'] = qm.apply(lambda row: q_cnt2(row), axis=1)
qm_pers = pd.merge(qm[qm['szp'].isna() == False], res[['inn','szp', 'snils']], how='left', on=['inn', 'szp']).drop_duplicates(['inn', 'snils'])
qm = qm[['inn', 'cnt1', 'cnt2']].groupby('inn').mean()
qm['ans'] = qm.cnt2 / qm.cnt1
qm[qm.ans > 3.7]

,cnt1,cnt2,ans
inn,,,
5.003022e+09,49303.563125,197885.372118,4.013612
5.051006e+09,53707.157857,201568.561429,3.753104
5.074046e+09,52921.512234,195996.043901,3.703523
7.701294e+09,78588.095238,294970.708095,3.753376
7.702062e+09,80552.333333,339784.222222,4.218180
...,...,...,...
7.743024e+09,51031.043519,197253.866987,3.865370
7.743086e+09,62953.558472,234563.122639,3.725971
7.743941e+09,57967.161358,227706.116358,3.928192


In [81]:
q01[q01['0.1'].isna()]

,0.1
inn,


In [82]:
print_df(qm, save_path + 'dezil')
print_df(qm_pers, save_path + 'dezil_pers')

In [83]:
cut_off = res[['inn', 'szp']].groupby('inn').count().rename(columns={'szp': 'cnt'})
cut_off.cnt = np.ceil(cut_off.cnt / 10)
cut_off = cut_off.to_dict()['cnt']
cut_off

{5003021368.0: 77.0,
 5003021495.0: 84.0,
 5003021640.0: 48.0,
 5003096290.0: 44.0,
 5030032182.0: 22.0,
 5030032376.0: 44.0,
 5051005503.0: 28.0,
 5051005623.0: 13.0,
 5051005670.0: 79.0,
 5074019220.0: 26.0,
 5074019252.0: 33.0,
 5074045703.0: 47.0,
 7701032028.0: 3.0,
 7701050524.0: 14.0,
 7701113894.0: 5.0,
 7701293580.0: 7.0,
 7701375995.0: 6.0,
 7701377657.0: 18.0,
 7701384943.0: 3.0,
 7701406844.0: 6.0,
 7701905508.0: 24.0,
 7702058364.0: 14.0,
 7702061938.0: 3.0,
 7702764296.0: 15.0,
 7702797527.0: 25.0,
 7702833359.0: 25.0,
 7703021039.0: 6.0,
 7703112310.0: 8.0,
 7703261626.0: 12.0,
 7703363890.0: 22.0,
 7703379868.0: 3.0,
 7703611221.0: 16.0,
 7703738796.0: 18.0,
 7703742922.0: 33.0,
 7703776689.0: 9.0,
 7703808740.0: 11.0,
 7704040242.0: 7.0,
 7704118121.0: 5.0,
 7704118139.0: 10.0,
 7704178709.0: 14.0,
 7704191153.0: 7.0,
 7704222919.0: 15.0,
 7704222926.0: 35.0,
 7704231631.0: 28.0,
 7704234760.0: 22.0,
 7704271031.0: 1.0,
 7704853583.0: 17.0,
 7705020295.0: 12.0,
 770504

In [84]:
ppl = res[['inn', 'szp']].sort_values('szp').dropna().groupby('inn')
keys = ppl.groups.keys()
val_1_7 = pd.DataFrame({'inn': [], 'val': []})
val_1_7

,inn,val


In [85]:
for key in keys:
    top = ppl.get_group(key).tail(int(cut_off[key]))['szp'].mean()
    bot = ppl.get_group(key).head(int(cut_off[key]))['szp'].mean()
    t_df = pd.DataFrame({'inn': [key], 'val': [top / bot]})
    val_1_7 = pd.concat([val_1_7, t_df])

In [86]:
print_df(val_1_7, save_path + 'val_1_7')

In [87]:
cnt1 = res[['inn', 'szp_teach']].groupby('inn').mean().rename(columns={'szp_teach':'cnt1'})
cnt2 = res[['inn', 'szp_teach']].groupby('inn').quantile(0.5).rename(columns={'szp_teach':'q'})
cnt2 = pd.merge(cnt2, res[['inn', 'snils', 'szp_teach']], how='right', on='inn')
teach_perc_out = cnt2
cnt2 = cnt2[cnt2.szp_teach <= cnt2.q][['inn', 'szp_teach']].groupby('inn').mean().rename(columns={'szp_teach':'cnt2'})
val_15 = pd.merge(cnt2, cnt1, how='left', on='inn')
val_15['val'] = val_15.cnt1 / val_15.cnt2
val_15[val_15.val > 1.3]

,cnt2,cnt1,val
inn,,,
7.727214e+09,128884.032706,171549.964383,1.331041


In [88]:
def qt01(row):
    if pd.isna(row['szp_teach']):
        return np.nan
    return row['szp_teach'] if row['szp_teach'] <= row['t01'] else np.nan

In [89]:
def qt09(row):
    if pd.isna(row['szp_teach']):
        return np.nan
    return row['szp_teach'] if row['szp_teach'] >= row['t09'] else np.nan

In [90]:
res.szp_teach.describe()

count     41442.000000
mean     127681.717291
std       27862.468463
min        5588.850000
25%      107864.955833
50%      125167.045833
75%      143611.154167
max      353671.556667
Name: szp_teach, dtype: float64

In [91]:
res[res.snils == '150-388-362 61']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach,szp_vosp
95200,7.734250e+09,150-388-362 61,203322.02,1.61,Учитель,183222.02,1.61,Учитель,176122.02,1.61,Учитель,187555.353333,187555.353333,187555.353333,NaN


In [92]:
t01 = res[res.szp_teach.isna() == False][['inn', 'szp_teach']].groupby('inn').quantile(0.1).rename(columns={'szp_teach': 't01'})
t09 = res[res.szp_teach.isna() == False][['inn', 'szp_teach']].groupby('inn').quantile(0.9).rename(columns={'szp_teach': 't09'})
qt = pd.merge(t01, t09, how='right', on='inn')
qt = pd.merge(qt, res[res.szp_teach.isna() == False][['inn', 'snils', 'szp_teach']], how='left', on='inn')
qt['qt01'] = qt.apply(lambda row: qt01(row), axis=1)
qt['qt09'] = qt.apply(lambda row: qt09(row), axis=1)
val_2_6 = qt[['inn', 'qt01', 'qt09']].groupby('inn').mean()
val_2_6['dezil'] = val_2_6.qt09 / val_2_6.qt01
val_2_6

,qt01,qt09,dezil
inn,,,
5.003021e+09,88544.489389,169925.546278,1.919098
5.003021e+09,84198.251857,185506.984714,2.203217
5.003022e+09,99426.023590,190845.823205,1.919476
5.003096e+09,93784.508021,200280.298437,2.135537
5.030032e+09,90973.248810,184630.170476,2.029500
...,...,...,...
7.743937e+09,69393.828889,153311.853333,2.209301
7.743939e+09,84414.840000,161996.636667,1.919054
7.743941e+09,102186.790769,206806.687564,2.023810


In [93]:
val_7[val_7.val < 1]

,cnt1,cnt2,val
inn,,,
5.003021e+09,296,291.0,0.983108
5.003021e+09,348,347.0,0.997126
5.030032e+09,66,65.0,0.984848
5.074019e+09,103,102.0,0.990291
5.074019e+09,86,85.0,0.988372
...,...,...,...
7.743021e+09,55,54.0,0.981818
7.743905e+09,97,95.0,0.979381
7.743937e+09,28,27.0,0.964286


In [94]:
val_2_6[val_2_6.dezil > 1.9]

,qt01,qt09,dezil
inn,,,
5.003021e+09,88544.489389,169925.546278,1.919098
5.003021e+09,84198.251857,185506.984714,2.203217
5.003022e+09,99426.023590,190845.823205,1.919476
5.003096e+09,93784.508021,200280.298437,2.135537
5.030032e+09,90973.248810,184630.170476,2.029500
...,...,...,...
7.743937e+09,69393.828889,153311.853333,2.209301
7.743939e+09,84414.840000,161996.636667,1.919054
7.743941e+09,102186.790769,206806.687564,2.023810


In [95]:
t01

,t01
inn,
5.003021e+09,97138.255000
5.003021e+09,91244.314500
5.003022e+09,107730.390000
5.003096e+09,97221.432667
5.030032e+09,100741.000000
...,...
7.743937e+09,98186.967000
7.743939e+09,86853.176000
7.743941e+09,109531.919000


In [96]:
t09

,t09
inn,
5.003021e+09,157143.286667
5.003021e+09,163889.401000
5.003022e+09,167894.650000
5.003096e+09,163329.844000
5.030032e+09,171630.670000
...,...
7.743937e+09,149820.903000
7.743939e+09,153103.166000
7.743941e+09,177801.034000


In [97]:
print_df(val_2_6, save_path + 'dezil_teach')

In [98]:
print_df(val_15, save_path + 'val_15_new')
print_df(teach_perc_out, save_path + 'val_15_ppl_new')

In [99]:
def summ_isp(row, months):
    sum = 0
    for month in months:
        if row['job_' + month] in isp:
            sum += row['sum_' + month]
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum

In [100]:
prep = create_full_res(months)

In [101]:
def sum_fot(row, months):
    sum = 0
    for month in months:
        if pd.isna(row['sum_' + month]) == False:
            sum += row['sum_' + month]
    return sum

In [102]:
prep['summ'] = prep.apply(lambda row: summ_isp(row, months), axis=1)
val_1_3_ppl = prep[(prep.summ.isna() == False) & (prep.summ > 0)]
val_1_3_ppl_2 = prep.copy()
val_1_3 = prep[['inn'] + ['sum_' + month for month in months]].groupby('inn').sum().reset_index()
val_1_3['sum'] = val_1_3.apply(lambda row: sum_fot(row, months), axis=1)
prep = prep[['inn', 'summ']].groupby('inn').sum().reset_index()
val_1_3 = pd.merge(prep[['inn', 'summ']], val_1_3[['inn', 'sum']], how='left', on='inn')
val_1_3['ans'] = val_1_3['summ'] / val_1_3['sum']
val_1_3

,inn,summ,sum,ans
0,5.003021e+09,2.354534e+08,2.747557e+08,0.856955
1,5.003021e+09,2.817875e+08,3.287882e+08,0.857049
2,5.003022e+09,2.122875e+08,2.462824e+08,0.861968
3,5.003096e+09,1.228814e+08,1.558069e+08,0.788678
4,5.030032e+09,7.016852e+07,8.427697e+07,0.832594
...,...,...,...,...
592,7.743941e+09,8.578093e+07,1.063156e+08,0.806852
593,9.705102e+09,0.000000e+00,1.461994e+08,0.000000
594,9.715207e+09,1.140669e+08,1.497550e+08,0.761690
595,9.715218e+09,5.538935e+07,6.823824e+07,0.811705


In [103]:
val_1_3[val_1_3.inn == 7725031305]

,inn,summ,sum,ans
315,7.725031e+09,41071805.44,67143122.08,0.611705


In [104]:
print_df(val_1_3_ppl_2, save_path + 'ppl_2_va_1_9')

In [105]:
val_1_3[val_1_3.inn == 7720258550]

,inn,summ,sum,ans
221,7.720259e+09,1688689.6,2130513.06,0.792621


In [106]:
val_1_3[val_1_3.ans < 0.75]

,inn,summ,sum,ans
12,7.701032e+09,10148386.74,1.453289e+07,0.698305
13,7.701051e+09,33712741.08,5.181362e+07,0.650654
14,7.701114e+09,9305905.42,1.411069e+07,0.659493
16,7.701376e+09,12350806.83,1.677062e+07,0.736455
19,7.701407e+09,13608545.72,2.017639e+07,0.674479
...,...,...,...,...
580,7.743086e+09,45383049.94,7.094170e+07,0.639723
581,7.743086e+09,65833900.81,9.984534e+07,0.659359
590,7.743937e+09,50179009.53,8.394682e+07,0.597748
593,9.705102e+09,0.00,1.461994e+08,0.000000


In [107]:
temp = create_full_res(months)

In [108]:
temp['szp_vosp'] = temp.apply(lambda row: szp_vosp(row, months), axis=1)
temp['szp_teach'] = temp.apply(lambda row: szp_teach(row, months), axis=1)

In [109]:
temp[temp.snils == '001-690-637 18']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach
4056,7.731174e+09,001-690-637 18,76546.94,1.0,Воспитатель,76546.94,1.0,Воспитатель,76546.94,1.0,Воспитатель,76546.94,NaN
174019,7.731230e+09,001-690-637 18,NaN,NaN,NaN,NaN,NaN,NaN,76546.94,1.0,Воспитатель,76546.94,NaN


In [110]:
def give_grp(row):
    if pd.isna(row['szp_teach']):
        return 0
    if row['szp_teach'] < 100000:
        return 1
    if row['szp_teach'] < 130000:
        return 2
    if row['szp_teach'] < 150000:
        return 3
    return 4

In [111]:
def give_grp2(row):
    if row['szp_teach'] < 80000:
        return 1
    if row['szp_teach'] > 150000:
        return 2
    return 0

In [112]:
temp['grp'] = temp.apply(lambda row: give_grp(row), axis=1)

In [113]:
temp['grp2'] = temp.apply(lambda row: give_grp2(row), axis=1)

In [114]:
temp[temp.grp2 == 2]

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach,grp,grp2
3,7.736615e+09,001-010-312 25,164214.30,1.0,Учитель,146547.14,1.0,Учитель,160599.30,1.0,Учитель,NaN,157120.246667,4,2
4,7.723170e+09,001-011-996 67,161441.63,1.0,Учитель,167905.34,1.0,Учитель,166752.15,1.0,Учитель,NaN,165366.373333,4,2
114,7.702833e+09,001-131-908 67,152088.02,1.0,Учитель,160320.62,1.0,Учитель,175707.24,1.0,Учитель,NaN,162705.293333,4,2
166,7.721151e+09,001-152-590 79,147882.00,1.0,Учитель,150251.40,1.0,Учитель,159644.78,1.0,Учитель,NaN,152592.726667,4,2
190,7.736051e+09,001-163-926 92,159502.90,0.9,Учитель,150559.81,0.9,Учитель,166829.14,0.9,Учитель,NaN,158963.950000,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175763,7.731243e+09,151-834-736 67,NaN,NaN,NaN,NaN,NaN,NaN,153568.01,1.0,Учитель,NaN,153568.010000,4,2
175766,7.731243e+09,151-908-182 62,NaN,NaN,NaN,NaN,NaN,NaN,152761.14,1.0,Учитель,NaN,152761.140000,4,2
175840,7.731243e+09,155-635-434 76,NaN,NaN,NaN,NaN,NaN,NaN,182367.50,1.0,Учитель,NaN,182367.500000,4,2
176199,7.723170e+09,175-284-645 98,NaN,NaN,NaN,NaN,NaN,NaN,221638.48,1.0,Учитель,NaN,221638.480000,4,2


In [115]:
temp[temp.grp == 0]

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach,grp,grp2
0,7.720259e+09,001-005-586 64,140968.88,1.0,Заведующий хозяйством,124968.88,1.0,Заведующий хозяйством,60434.74,1.0,Заведующий хозяйством,NaN,NaN,0,0
5,7.727124e+09,001-017-453 65,90135.84,1.0,Документовед,87135.84,1.0,Секретарь руководителя,86627.92,1.0,Секретарь руководителя,NaN,NaN,0,0
7,7.701051e+09,001-020-587 55,112852.50,1.0,Старший методист,81028.79,1.0,Старший методист,122370.00,1.0,Старший методист,NaN,NaN,0,0
8,7.729776e+09,001-020-711 41,65993.00,1.0,Воспитатель,60020.00,1.0,Воспитатель,68000.00,1.0,Воспитатель,64671.0,NaN,0,0
9,7.723356e+09,001-021-199 51,66502.39,1.0,Преподаватель,61918.39,1.0,Преподаватель,59040.96,1.0,Преподаватель,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176757,7.734409e+09,215-089-180 55,NaN,NaN,NaN,NaN,NaN,NaN,25039.34,1.0,Специалист,NaN,NaN,0,0
176758,7.736615e+09,215-093-175 39,NaN,NaN,NaN,NaN,NaN,NaN,38178.00,1.0,Уборщик территории,NaN,NaN,0,0
176759,7.743021e+09,215-094-406 39,NaN,NaN,NaN,NaN,NaN,NaN,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN,0,0
176760,7.736051e+09,215-120-882 18,NaN,NaN,NaN,NaN,NaN,NaN,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...,NaN,NaN,0,0


In [116]:
print_df(temp, save_path + 'Учителя по группам')

In [117]:
temp[['grp', 'szp_teach']].groupby('grp').mean()

,szp_teach
grp,
0,NaN
1,85445.003737
2,115042.855962
3,139043.918370
4,170377.186003


In [118]:
temp[['grp2', 'szp_teach']].groupby('grp2').mean()

,szp_teach
grp2,
0,114677.789469
1,66874.387804
2,170377.186003


In [119]:
temp[['grp'] + ['stv_' + month for month in months]].groupby('grp').mean()

,stv_jan,stv_feb,stv_mar
grp,,,
0,0.996425,0.995939,0.995493
1,1.050831,1.050247,1.049498
2,1.059312,1.059321,1.059627
3,1.064571,1.064928,1.065099
4,1.065588,1.065423,1.065466


In [120]:
temp[temp.inn == 7720258550]

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp_vosp,szp_teach,grp,grp2
56323,7.720259e+09,037-829-622 90,132119.81,1.00,Заместитель директора,309703.65,1.000000,Заместитель директора,NaN,NaN,NaN,NaN,NaN,0,0
129528,7.720259e+09,139-912-912 93,84213.54,1.17,Учитель,87813.54,1.170000,Учитель,84113.54,1.170000,Учитель,NaN,85380.206667,1,0
139913,7.720259e+09,149-672-801 06,84133.14,1.39,Учитель,55819.15,1.390000,Учитель,84133.14,1.390000,Учитель,NaN,74695.143333,1,1
144286,7.720259e+09,154-173-576 64,68754.74,1.22,Учитель,71965.84,1.111111,Учитель,84145.90,1.111111,Учитель,NaN,74955.493333,1,1
148603,7.720259e+09,159-934-072 11,84146.62,1.00,Учитель,70922.47,1.050000,Учитель,78535.34,1.050000,Учитель,NaN,77868.143333,1,1
149928,7.720259e+09,162-011-849 20,98727.59,1.11,Учитель,84131.56,1.110000,Учитель,66104.80,1.110000,Учитель,NaN,82987.983333,1,0
150749,7.720259e+09,163-099-703 82,80352.86,1.67,Учитель,84131.84,1.670000,Учитель,84131.84,1.670000,Учитель,NaN,82872.180000,1,0
157281,7.720259e+09,173-181-439 62,84133.94,1.05,Учитель,84144.27,1.050000,Учитель,84133.94,1.050000,Учитель,NaN,84137.383333,1,0


In [121]:
print_df(temp[temp.inn == 7720258550], 'sch_check')

In [122]:
val_1_3[val_1_3.ans < 0.75]

,inn,summ,sum,ans
12,7.701032e+09,10148386.74,1.453289e+07,0.698305
13,7.701051e+09,33712741.08,5.181362e+07,0.650654
14,7.701114e+09,9305905.42,1.411069e+07,0.659493
16,7.701376e+09,12350806.83,1.677062e+07,0.736455
19,7.701407e+09,13608545.72,2.017639e+07,0.674479
...,...,...,...,...
580,7.743086e+09,45383049.94,7.094170e+07,0.639723
581,7.743086e+09,65833900.81,9.984534e+07,0.659359
590,7.743937e+09,50179009.53,8.394682e+07,0.597748
593,9.705102e+09,0.00,1.461994e+08,0.000000


In [123]:
print_df(val_1_3, 'val_1_3')
print_df(val_1_3_ppl, 'val_1_3_ppl')

In [124]:
len(val_1_3[val_1_3.ans < 0.75])

129

In [125]:
res[res.snils == '016-412-327 06']

,inn,snils,sum_jan,stv_jan,job_jan,sum_feb,stv_feb,job_feb,sum_mar,stv_mar,job_mar,szp,szp_ped,szp_teach,szp_vosp
15687,7.712022e+09,016-412-327 06,129341.55,1.05,Учитель,139859.09,1.05,Учитель,129431.09,1.05,Учитель,132877.243333,132877.243333,132877.243333,NaN


In [126]:
out = res[['inn', 'szp_ped']].groupby('inn').count().rename(columns={'szp_ped': 'Количество педов'})
out

,Количество педов
inn,
5.003021e+09,641
5.003021e+09,702
5.003022e+09,327
5.003096e+09,340
5.030032e+09,170
...,...
7.743941e+09,205
9.705102e+09,0
9.715207e+09,262


In [127]:
res = load_data('mar')

In [128]:
res

,inn,snils,sum_mar,stv_mar,job_mar
1,7.718310e+09,001-005-786 70,85809.12,1.0,Учитель
3,7.736615e+09,001-010-312 25,160599.30,1.0,Учитель
4,7.723170e+09,001-011-996 67,166752.15,1.0,Учитель
5,7.727124e+09,001-017-453 65,86627.92,1.0,Секретарь руководителя
6,7.733089e+09,001-017-484 72,86787.10,1.0,Учитель
...,...,...,...,...,...
166711,7.734409e+09,215-089-180 55,25039.34,1.0,Специалист
166712,7.736615e+09,215-093-175 39,38178.00,1.0,Уборщик территории
166713,7.743021e+09,215-094-406 39,10015.73,1.0,Рабочий по комплексному обслуживанию и ремонту...
166714,7.736051e+09,215-120-882 18,13916.40,1.0,Рабочий по комплексному обслуживанию и ремонту...


In [129]:
res[res.snils == '001-697-876 59']

,inn,snils,sum_mar,stv_mar,job_mar
4111,7.736111e+09,001-697-876 59,256222.11,1.0,Художественный руководитель


In [130]:
#меньше 84100 учителя март
min_teach = res[res.job_mar == 'Учитель'][['inn', 'snils', 'sum_mar']].groupby('snils').max().reset_index()
print_df(min_teach[min_teach.sum_mar < 84100], 'min_teach')

In [131]:
#меньше 54500 не педы
min_nped = res[res.job_mar.isin(ped) == False][['inn', 'snils', 'sum_mar']].groupby('snils').max().reset_index()
print_df(min_nped[min_nped.sum_mar < 54500], 'min_nped')

In [132]:
out = df[df.job.isin(['Учитель'])].drop_duplicates(['inn', 'snils'])[['inn', 'snils']].groupby('inn').count().rename(columns={'snils':'Количество педов'})

AttributeError: 'DataFrame' object has no attribute 'job'

In [ ]:
out
wb = Workbook()
ws = wb.active
for  i in df_to_row(out, header=True):
    ws.append(i)
wb.save('teach.xlsx')

In [ ]:
school_sum = sample[['inn', 'sum']].groupby('inn').sum()

In [ ]:
school_sum

In [ ]:
s = sums.groupby(['snils']).count().reset_index()
wb = Workbook()
ws = wb.active
for  i in df_to_row(s, header=True):
    ws.append(i)
wb.save('test.xlsx')